# This notebook serves as an example for processing the binding affinity data from SKEMPI. 

In [163]:
from Bio.PDB import *
import pandas as pd
import warnings
from Bio import BiopythonWarning
warnings.simplefilter('ignore', BiopythonWarning)
import csv
import numpy as np
from Bio.PDB.Polypeptide import three_to_one, one_to_three                           6
import math


## Function for loading SKEMPI.csv file

In [164]:
def load_pd(fName):
    df = pd.read_csv(fName, sep = ';')
    df = df.dropna(subset = ['Affinity_mut_parsed', 'Affinity_wt_parsed', 'Temperature'])
    return df


## Function for generating the sequence from PDB ID. Note that you need to have a folder (./skempi_v2/PDBs/) of pre-downloaded PDB files. 

In [165]:
def get_seq(ent_name, mutation_pdb, chains, affinity, temp, seq_dict):
#    print(mutation_pdb)
    chain_ID, residue_ID, r_w, r_m = mutation_pdb[1], mutation_pdb[2:-1], mutation_pdb[0], mutation_pdb[-1]
    # print('chain_ID, residue_ID, r_w, r_m',chain_ID, residue_ID, r_w, r_m)
    
        
    parser = PDBParser()
    structure = parser.get_structure(ent_name, './skempi_v2/PDBs/' + ent_name + '.pdb')
#    print(structure) # this print structure id for first 10 elements <Structure id=1CSE>
    # print(len(structure)) # the structure length are all 1
    assert(len(structure) == 1)
    model = structure[0]

#     for elem in model.get_chains():
#         print(elem)
    w_seq0 = ''
    
    for each_chain in chains[0]:
    #    print(each_chain,)
        chain0 = model[each_chain]
        ppb = PPBuilder()
        for pp in ppb.build_peptides(chain0): # one chain could have multiple sequence seperatly measured
        #    print(pp)
        #    print('Ch0',pp.get_sequence())
            w_seq0 += pp.get_sequence() # record chain sequencing for first chain in each sample protein

    w_seq1 = ''
    for each_chain in chains[1]:
#         print(each_chain,)
        chain1 = model[each_chain]
        for pp in ppb.build_peptides(chain1):
    #         print('Ch1',pp.get_sequence())
            w_seq1 += pp.get_sequence()

    w_name0 = mut_name0 = ent_name+'_'+chains[0]
    w_name1 = mut_name1 = ent_name+'_'+chains[1]
#    print("Wildtype", w_name0, w_name1)
#    print(w_seq0)
#    print(w_seq1)
    
    if w_name0 not in seq_dict:
        seq_dict[w_name0] = w_seq0
    if w_name1 not in seq_dict:
        seq_dict[w_name1] = w_seq1
#    print(seq_dict) # why exists 'leu38gly'?? generate from following

#     Mutation
    mut_chain = model[chain_ID]
    
    if residue_ID.isdigit():
#         print(residue_ID)
        residue = mut_chain[int(residue_ID)]
    else:
#         print(residue_ID[:-1], residue_ID[-1])
        residue = mut_chain[(' ', int(residue_ID[:-1]), residue_ID[-1].upper())]

#    print(residue.resname, three_to_one(residue.resname), one_to_three(r_m))
    assert(three_to_one(residue.resname) == r_w)

    residue.resname = one_to_three(r_m)
    
    m_seq0 = ''
    m_seq1 = ''

#     chain_ID, residue_ID, r_w, r_m = mutation_pdb[1], mutation_pdb[2:-1], mutation_pdb[0], mutation_pdb[-1]
    
    for each_chain in chains[0]:
        chain0 = model[each_chain]
#        print(chain_ID, chain0)
        if chain_ID == chain0.id:
    #        print("mutation on 1st chain")
#             mut_name0 = ent_name+'_'+chains[0] + '_' + mutation_pdb
            mut_name0 = ent_name+'_'+chains[0] + '_' + one_to_three(r_w) + str(residue_ID) + one_to_three(r_m)
    #        print(mut_name0)

#             m_seq0 = ''
            for pp in ppb.build_peptides(chain0):
                m_seq0 += pp.get_sequence()
        else:
            for pp in ppb.build_peptides(chain0):
                m_seq0 += pp.get_sequence()
    
    for each_chain in chains[1]:
        chain1 = model[each_chain]
#        print(chain_ID, chain1)
        if chain_ID == chain1.id:
#            print("mutation on 2nd chain")
#             mut_name1 = ent_name+'_'+chains[1] + '_' + mutation_pdb
            mut_name1 = ent_name+'_'+chains[1] + '_' + one_to_three(r_w) + str(residue_ID) + one_to_three(r_m)
#            print(mut_name1)
#             m_seq0 = ''
            for pp in ppb.build_peptides(chain1):
                m_seq1 += pp.get_sequence()
        else:
            for pp in ppb.build_peptides(chain1):
                m_seq1 += pp.get_sequence()

    if mut_name0 not in seq_dict:
        seq_dict[mut_name0] = m_seq0
    if mut_name1 not in seq_dict:
        seq_dict[mut_name1] = m_seq1
#    print(seq_dict)    
    
    dG_w =  (8.314/4184)*(float(temp)) * math.log(float(affinity[1]))
    dG_m = (8.314/4184)*(float(temp)) * math.log(float(affinity[0]))
    #print("dG_w, dG_m, ddG",dG_w, dG_m, dG_m - dG_w)
#     fout.write(w_name0, w_name1,mut_name0, mut_name1,dG_w, dG_m, dG_m - dG_w)
    print(w_name0, w_name1,mut_name0, mut_name1,dG_w, dG_m, dG_m - dG_w, file=open("./skempi_v2.singlemut.ddg.score.txt", "a"))
    
#     print(w_name0, w_name1,mut_name0, mut_name1,dG_w, dG_m, dG_m - dG_w, file=open("./skempi_v2.mut4.ddg.score.txt", "a"))
    return seq_dict



The binding free energy is calculated by equation:

$\Delta G = -RT ln(K_d)$
where $K_d$ is binding affinity.

*ref : Hu, J., Zhou, L., Li, B. et al. Improve hot region prediction by analyzing different machine learning algorithms. BMC Bioinformatics 22, 522 (2021). https://doi.org/10.1186/s12859-021-04420-0*

## Function for processing the binding affinity scores

In [166]:
# download_all_pdb(ppi)
def get_score(ent_name, mutation_pdb, chains, affinity, temp):
    
    chain_ID, residue_ID, r_w, r_m = mutation_pdb[1], mutation_pdb[2:-1], mutation_pdb[0], mutation_pdb[-1]
    #print('chain_ID, residue_ID, r_w, r_m',chain_ID, residue_ID, r_w, r_m)
    
        
    w_name0 = mut_name0 = ent_name+'_'+chains[0]
    w_name1 = mut_name1 = ent_name+'_'+chains[1]
#     print("Wildtype", w_name0, w_name1)
    
#     Mutation
    
    for each_chain in chains[0]:
#         print(chain_ID, chain0)
        if chain_ID == each_chain:
#             print("mutation on 1st chain")
            mut_name0 = ent_name+'_'+chains[0] + '_' + one_to_three(r_w) + str(residue_ID) + one_to_three(r_m)
    
    for each_chain in chains[1]:
#         print(chain_ID, chain1)
        if chain_ID == each_chain:
#             print("mutation on 2nd chain")
            mut_name1 = ent_name+'_'+chains[1] + '_' + one_to_three(r_w) + str(residue_ID) + one_to_three(r_m)

    dG_w =  (8.314/4184)*(float(temp)) * math.log(float(affinity[1]))
    dG_m = (8.314/4184)*(float(temp)) * math.log(float(affinity[0]))
    #print(w_name0, w_name1,mut_name0, mut_name1,dG_w, dG_m, dG_m - dG_w)
#    print(w_name0, w_name1,mut_name0, mut_name1,dG_w, dG_m, dG_m - dG_w, file=open("./skempi_v2.singlemut.ddg.score.txt", "a"))
    return seq_dict


## Please provide the SKEMPI file here. In our demo, we choose to use SKEMPI version 2 downloaded from https://life.bsc.es/pid/skempi2/database/index

In [167]:
skepiIn = "./skempi_v2/skempi_v2.csv"
ppi = load_pd(skepiIn)
print("Size of the data:", ppi.shape)

ppi = np.array(ppi)
# print(ppi)

# process temperature at 13 columns
select_N = 10
print("Top " + str(select_N) + " samples are selected for demo purpose.")
for i in range(select_N):
    elem = ppi[i, 13].strip()
    ppi[i, 13] = elem[:3] # some termperature followed by 298(assumed), [:3] keeps only number

Size of the data: (6794, 29)
Top 10 samples are selected for demo purpose.


## The main processing part. The processed file will be saved. 

#### Two output : "./skempi_v2/singlemut.ddg.score.txt " denotes the processed score file, and "./skempi_v2.singlemut.mut4.seq.txt" denotes the processed sequence file.
The processed score file contains 7 fields for each row: Protein_wild_1, Protein_wild_1, Protein_mutant_1, Protein_mutant_2, $\Delta{G_w}$, $\Delta{G_m}$, $\Delta\Delta{G}$

In [168]:
mult_mutation = 0
seq_dict = {}


cnt_single_chain = cnt_both_chain = 0

f1 = open("./skempi_v2/singlemut.ddg.score.txt", "w")
f2 = open("./skempi_v2.singlemut.mut4.seq.txt", "w")
print(ppi.shape)

for i in range(select_N):
# for i in range(10):
#     print(ppi[i,:])
    ent_name = ppi[i,0][:4]
#     .lower()
    chains = ppi[i,0].split('_')[1:]
    assert(len(chains) == 2)
#     print(i, len(ppi))
    affinity = ppi[i,(7,9)]
    temp = ppi[i,13]
    #print(i, ppi[i, :3], temp, affinity)
    mutations_pdb = ppi[i][2].split(',') # could be multiple mutation in one row
#     if len(mutations_pdb) == 1:
    get_seq(ent_name, mutations_pdb[0], chains, affinity, temp, seq_dict)
    # get_score(ent_name, mutations_pdb[0], chains, affinity, temp) # get seq function also produce score

for elem in seq_dict:
    print(elem, seq_dict[elem], file=open("./skempi_v2.singlemut.mut4.seq.txt", "a"))
    


(6794, 29)


After `get seq` function, `seq_dic` consists of expected mutation in the formate of :

{'1CSE_E': Seq('AQTVPYGIPLIKADKVQAQGFKGANVKVAVLDTGIQASHPDLNVVGGASFVAGE...AAQ'), 
'1CSE_I': Seq('KSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTLDLRYNRVRVFYNPGTN...HVG'), 
'1CSE_I_LEU38GLY': Seq('KSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTGDLRYNRVRVFYNPGTN...HVG'), 
'1CSE_I_LEU38SER': Seq('KSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTSDLRYNRVRVFYNPGTN...HVG'), 
'1CSE_I_LEU38PRO': Seq('KSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTPDLRYNRVRVFYNPGTN...HVG'), 
'1CSE_I_LEU38ILE': Seq('KSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTIDLRYNRVRVFYNPGTN...HVG'), 
'1CSE_I_LEU38ASP': Seq('KSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTDDLRYNRVRVFYNPGTN...HVG'), 
'1CSE_I_LEU38GLU': Seq('KSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTEDLRYNRVRVFYNPGTN...HVG'), 
'1ACB_E': Seq('CGVPAIQPVLSGLIVNGEEAVPGSWPWQVSLQDKTGFHFCGGSLINENWVVTAA...AAN'), 
'1ACB_I': Seq('KSFPEVVGKTVDQAREYFTLHYPQYDVYFLPEGSPVTLDLRYNRVRVFYNPGTN...HVG'), 
'1ACB_I_LEU38GLY': Seq('KSFPEVVGKTVDQAREYFTLHYPQYDVYFLPEGSPVTGDLRYNRVRVFYNPGTN...HVG'), 
'1ACB_I_LEU38SER': Seq('KSFPEVVGKTVDQAREYFTLHYPQYDVYFLPEGSPVTSDLRYNRVRVFYNPGTN...HVG'), 
'1ACB_I_LEU38PRO': Seq('KSFPEVVGKTVDQAREYFTLHYPQYDVYFLPEGSPVTPDLRYNRVRVFYNPGTN...HVG'), 
'1ACB_I_LEU38ILE': Seq('KSFPEVVGKTVDQAREYFTLHYPQYDVYFLPEGSPVTIDLRYNRVRVFYNPGTN...HVG')}

Although sequencing data hasn't change, but the mutation are marked. 